# Financial Report Generation

<a href="https://colab.research.google.com/github/run-llama/llamacloud-demo/blob/main/examples/report_generation/report_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook we show you how to perform financial report generation with LlamaCloud consisting of text and tables, given an existing bank of reports.

LlamaCloud provides advanced retrieval endpoints allowing you to fetch chunk and document-level context from complex financial reports consisting of text, tables, and sometimes images/diagrams.

We build an agentic workflow on top of LlamaCloud consisting of researcher and writer steps in order to generate the final response.

![](https://github.com/run-llama/llamacloud-demo/blob/main/examples/report_generation/financial_report_generation_img.png?raw=1)

## Setup

Install core packages, download 10k files from Apple and Tesla.

You will need to upload these documents to LlamaCloud. For best results, we recommend:
- Setting Parse settings to "Accurate" mode, "Premium" mode, or "3rd Party multimodal"
- Setting the "Segmentation Configuration" to "Page" and the "Chunking Configuration" to None. This will give you page-level chunks.

In [1]:
!pip install llama-index
!pip install llama-index-core
!pip install llama-index-embeddings-openai
!pip install llama-index-question-gen-openai
!pip install llama-index-postprocessor-flag-embedding-reranker
!pip install git+https://github.com/FlagOpen/FlagEmbedding.git
!pip install llama-parse

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.3/454.3 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.0 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4
  Cloning https://github.com/FlagOpen/FlagEmbedding.git to /tmp/pip-req-build-t3f_d2cz
  Running command git clone --filter=blob:none --quiet https://github.com/FlagOpen/FlagEmbedding.git /tmp/pip-req-build-t3f_d2cz
  Resolved https://github.com/FlagOpen/FlagEmbedding.git to commit 049882837fe3cffaa47d07fd33f153d5cfca6050
  Preparing metadata (setup.py) ... done
     

In [2]:
!mkdir data
# download Apple
!wget "https://s2.q4cdn.com/470004039/files/doc_earnings/2023/q4/filing/_10-K-Q4-2023-As-Filed.pdf" -O data/apple_2023.pdf
!wget "https://s2.q4cdn.com/470004039/files/doc_financials/2022/q4/_10-K-2022-(As-Filed).pdf" -O data/apple_2022.pdf
!wget "https://s2.q4cdn.com/470004039/files/doc_financials/2021/q4/_10-K-2021-(As-Filed).pdf" -O data/apple_2021.pdf
!wget "https://s2.q4cdn.com/470004039/files/doc_financials/2020/ar/_10-K-2020-(As-Filed).pdf" -O data/apple_2020.pdf
!wget "https://www.dropbox.com/scl/fi/i6vk884ggtq382mu3whfz/apple_2019_10k.pdf?rlkey=eudxh3muxh7kop43ov4bgaj5i&dl=1" -O data/apple_2019.pdf

# download Tesla
!wget "https://ir.tesla.com/_flysystem/s3/sec/000162828024002390/tsla-20231231-gen.pdf" -O data/tesla_2023.pdf
!wget "https://ir.tesla.com/_flysystem/s3/sec/000095017023001409/tsla-20221231-gen.pdf" -O data/tesla_2022.pdf
!wget "https://www.dropbox.com/scl/fi/ptk83fmye7lqr7pz9r6dm/tesla_2021_10k.pdf?rlkey=24kxixeajbw9nru1sd6tg3bye&dl=1" -O data/tesla_2021.pdf
!wget "https://ir.tesla.com/_flysystem/s3/sec/000156459021004599/tsla-10k_20201231-gen.pdf" -O data/tesla_2020.pdf
!wget "https://ir.tesla.com/_flysystem/s3/sec/000156459020004475/tsla-10k_20191231-gen_0.pdf" -O data/tesla_2019.pdf

--2025-01-03 15:34:32--  https://s2.q4cdn.com/470004039/files/doc_earnings/2023/q4/filing/_10-K-Q4-2023-As-Filed.pdf
Resolving s2.q4cdn.com (s2.q4cdn.com)... 68.70.205.2, 68.70.205.4, 68.70.205.3, ...
Connecting to s2.q4cdn.com (s2.q4cdn.com)|68.70.205.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 714094 (697K) [application/pdf]
Saving to: ‘data/apple_2023.pdf’

data/apple_2023.pdf 100%[===================>] 697.36K  --.-KB/s    in 0.03s   

2025-01-03 15:34:32 (21.2 MB/s) - ‘data/apple_2023.pdf’ saved [714094/714094]

--2025-01-03 15:34:32--  https://s2.q4cdn.com/470004039/files/doc_financials/2022/q4/_10-K-2022-(As-Filed).pdf
Resolving s2.q4cdn.com (s2.q4cdn.com)... 68.70.205.2, 68.70.205.4, 68.70.205.3, ...
Connecting to s2.q4cdn.com (s2.q4cdn.com)|68.70.205.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 729516 (712K) [application/pdf]
Saving to: ‘data/apple_2022.pdf’

data/apple_2022.pdf 100%[===================>] 712.42K  

We set the tokenizer to be gpt-4o specific. Some of our workflows involving cramming as much context into the prompt, and to make this work robustly without context overflow errors, we will want to make sure our tokenizer is accurate.

In [3]:
from llama_index.core import set_global_tokenizer
import tiktoken

set_global_tokenizer(tiktoken.encoding_for_model("gpt-4o").encode)

In [4]:
# llama-parse is async-first, running the async code in a notebook requires the use of nest_asyncio
import nest_asyncio
nest_asyncio.apply()

In [5]:
from google.colab import userdata
import os

api_key = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = api_key

In [ ]:
import os
# API access to llama-cloud
os.environ["LLAMA_CLOUD_API_KEY"] = "llx-IH0KAZxTywVVSG9ysBC1dkpM2oeGCCfFO1jtzfnUfoqnnvLI"

In [ ]:
### setup embedding/LLM model
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

embed_model = OpenAIEmbedding(model="text-embedding-3-large")
llm = OpenAI(model="gpt-4o-mini")

Settings.embed_model = embed_model
Settings.llm = llm

## Load Documents into LlamaCloud

The first order of business is to download the 5 Apple and Tesla 10Ks and upload them into LlamaCloud.

You can easily do this by creating a pipeline and uploading docs via the "Files" mode.

After this is done, proceed to the next section.

## Define LlamaCloud File/Chunk Retriever over Documents

In this section we define both a file-level and chunk-level LlamaCloud Retriever over these documents.

The file-level LlamaCloud retriever returns entire documents with a `files_top_k`. There are two retrieval modes:
- `files_via_content`: Retrieve top-k chunks, dereference into source files. Use a weighted average heuristic to determine the top files to return.
- `files_via_metadata`: Use an LLM to analyze the metadata of each file, and determine the top files that are most relevant to the query.

The chunk-level LlamaCloud retriever is our default retriever that returns chunks via hybrid search + reranking.

In [ ]:
from llama_index.indices.managed.llama_cloud import LlamaCloudIndex
import os

index = LlamaCloudIndex(
  name="apple_tesla_demo_2",
  project_name="llamacloud_demo",
  api_key=os.environ["LLAMA_CLOUD_API_KEY"]
)

#### Define File Retriever

In this section we define the file-level retriever. By default we use `retrieval_mode="files_via_content"`, but you can also change it to `files_via_metadata`.

In [ ]:
doc_retriever = index.as_retriever(
    retrieval_mode="files_via_content",
    files_top_k=1
)

In [ ]:
nodes = doc_retriever.retrieve("Give me a summary of Tesla in 2019")

#### Define chunk retriever

The chunk-level retriever does vector search with a final reranked set of `rerank_top_n=5`.

In [ ]:
chunk_retriever = index.as_retriever(
    retrieval_mode="chunks",
    rerank_top_n=5
)

#### Define Retriever Tools

Wrap these with Python functions into tool objects - these will directly be used by the LLM.

In [ ]:
from llama_index.core.tools import FunctionTool
from llama_index.core.schema import NodeWithScore
from typing import List

# function tools
def chunk_retriever_fn(query: str) -> List[NodeWithScore]:
    """Retrieves a small set of relevant document chunks from the corpus.

    ONLY use for research questions that want to look up specific facts from the knowledge corpus,
    and don't need entire documents.

    """
    return chunk_retriever.retrieve(query)

def doc_retriever_fn(query: str) -> float:
    """Document retriever that retrieves entire documents from the corpus.

    ONLY use for research questions that may require searching over entire research reports.

    Will be slower and more expensive than chunk-level retrieval but may be necessary.
    """
    return doc_retriever.retrieve(query)

chunk_retriever_tool = FunctionTool.from_defaults(fn=chunk_retriever_fn)
doc_retriever_tool = FunctionTool.from_defaults(fn=doc_retriever_fn)

## Build a Report Generation Workflow

Now that we've defined the retrievers, we're ready to build the report generation workflow.

The workflow contains roughly the following steps:

1. **Research Gathering**: Perform a function calling loop where the agent tries to reason about what tool to call (chunk-level or document-level retrieval) in order to gather more information. All information is shared to a dictionary that is propagated throughout each step. The tools return an indication of the type of information returned to the agent. After the agent feels like it's gathered enough information, move on to the next phase.
2. **Report Generation**: Generate a research report given the pooled research. For now, try to stuff as much information into the context window through the summary index.

This implementation is inspired by our [Function Calling Agent](https://docs.llamaindex.ai/en/stable/examples/workflow/function_calling_agent/) workflow implementation.

In [ ]:
from llama_index.llms.openai import OpenAI
from pydantic import BaseModel, Field
from typing import List, Tuple
import pandas as pd
from IPython.display import display, Markdown


class TextBlock(BaseModel):
    """Text block."""

    text: str = Field(..., description="The text for this block.")


class TableBlock(BaseModel):
    """Image block."""

    caption: str = Field(..., description="Caption of the table.")
    col_names: List[str] = Field(..., description="Names of the columns.")
    rows: List[Tuple] = Field(
        ...,
        description=(
            "List of rows. Each row is a data entry tuple, "
            "where each element of the tuple corresponds positionally to the column name."
        )
    )

    def to_df(self) -> pd.DataFrame:
        """To dataframe."""
        df = pd.DataFrame(self.rows, columns=self.col_names)
        df.style.set_caption(self.caption)
        return df


class ReportOutput(BaseModel):
    """Data model for a report.

    Can contain a mix of text and table blocks. Use table blocks to present any quantitative metrics and comparisons.

    """

    blocks: List[TextBlock | TableBlock] = Field(
        ..., description="A list of text and table blocks."
    )

    def render(self) -> None:
        """Render as formatted text within a jupyter notebook."""
        for b in self.blocks:
            if isinstance(b, TextBlock):
                display(Markdown(b.text))
            else:
                display(b.to_df())


report_gen_system_prompt = """\
You are a report generation assistant tasked with producing a well-formatted report given parsed context.
You will be given context from one or more reports that take the form of parsed text + tables
You are responsible for producing a report with interleaving text and tables - in the format of interleaving text and "table" blocks.

Make sure the report is detailed with a lot of textual explanations especially if tables are given.

You MUST output your response as a tool call in order to adhere to the required output format. Do NOT give back normal text.

Here is an example of a toy valid tool call - note the text and table block:
```
{
    "blocks": [
        {
            "text": "A report on cities"
        },
        {
            "caption": "Comparison of CityA vs. CityB",
            "col_names": [
              "",
              "Population",
              "Country",
            ],
            "rows": [
              [
                "CityA",
                "1,000,000",
                "USA"
              ],
              [
                "CityB",
                "2,000,000",
                "Mexico"
              ]
            ]
        }
    ]
}
```
"""

report_gen_llm = OpenAI(model="gpt-4o", max_tokens=2048, system_prompt=report_gen_system_prompt)
report_gen_sllm = report_gen_llm.as_structured_llm(output_cls=ReportOutput)

In [ ]:
from llama_index.core.workflow import Workflow

from typing import Any, List
from operator import itemgetter

from llama_index.core.llms.function_calling import FunctionCallingLLM
from llama_index.core.llms.structured_llm import StructuredLLM
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.llms import ChatMessage
from llama_index.core.tools.types import BaseTool
from llama_index.core.tools import ToolSelection
from llama_index.core.workflow import Workflow, StartEvent, StopEvent, Context, step
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.response_synthesizers import TreeSummarize, CompactAndRefine
from llama_index.core.workflow import Event


class InputEvent(Event):
    input: List[ChatMessage]


class ChunkRetrievalEvent(Event):
    tool_call: ToolSelection


class DocRetrievalEvent(Event):
    tool_call: ToolSelection


class ReportGenerationEvent(Event):
    pass


class ReportGenerationAgent(Workflow):
    """Report generation agent."""

    def __init__(
        self,
        chunk_retriever_tool: BaseTool,
        doc_retriever_tool: BaseTool,
        llm: FunctionCallingLLM | None = None,
        report_gen_sllm: StructuredLLM | None = None,
        **kwargs: Any,
    ) -> None:
        super().__init__(**kwargs)
        self.chunk_retriever_tool = chunk_retriever_tool
        self.doc_retriever_tool = doc_retriever_tool

        self.llm = llm or OpenAI()
        self.summarizer = CompactAndRefine(llm=self.llm)
        assert self.llm.metadata.is_function_calling_model

        self.report_gen_sllm = report_gen_sllm or self.llm.as_structured_llm(
            ReportOutput, system_prompt=report_gen_system_prompt
        )
        self.report_gen_summarizer = TreeSummarize(llm=self.report_gen_sllm)

        self.memory = ChatMemoryBuffer.from_defaults(llm=llm)
        self.sources = []

    @step(pass_context=True)
    async def prepare_chat_history(self, ctx: Context, ev: StartEvent) -> InputEvent:
        # clear sources
        self.sources = []

        ctx.data["stored_chunks"] = []
        ctx.data["query"] = ev.input

        # get user input
        user_input = ev.input
        user_msg = ChatMessage(role="user", content=user_input)
        self.memory.put(user_msg)

        # get chat history
        chat_history = self.memory.get()
        return InputEvent(input=chat_history)

    @step(pass_context=True)
    async def handle_llm_input(
        self, ctx: Context, ev: InputEvent
    ) -> ChunkRetrievalEvent | DocRetrievalEvent | ReportGenerationEvent | StopEvent:
        chat_history = ev.input

        response = await self.llm.achat_with_tools(
            [self.chunk_retriever_tool, self.doc_retriever_tool],
            chat_history=chat_history,
        )
        self.memory.put(response.message)

        tool_calls = self.llm.get_tool_calls_from_response(
            response, error_on_no_tool_call=False
        )
        for tool_call in tool_calls:
            if self._verbose:
                print(f"Tool call: {tool_call}")
        if not tool_calls:
            # all the content should be stored in the context, so just pass along input
            return ReportGenerationEvent(input=ev.input)

        for tool_call in tool_calls:
            if tool_call.tool_name == self.chunk_retriever_tool.metadata.name:
                return ChunkRetrievalEvent(tool_call=tool_call)
            elif tool_call.tool_name == self.doc_retriever_tool.metadata.name:
                return DocRetrievalEvent(tool_call=tool_call)
            else:
                return StopEvent(result={"response": "Invalid tool."})

    @step(pass_context=True)
    async def handle_retrieval(
        self, ctx: Context, ev: ChunkRetrievalEvent | DocRetrievalEvent
    ) -> InputEvent:
        """Handle retrieval.

        Store retrieved chunks, and go back to agent reasoning loop.

        """
        query = ev.tool_call.tool_kwargs["query"]
        if isinstance(ev, ChunkRetrievalEvent):
            retrieved_chunks = self.chunk_retriever_tool(query).raw_output
        else:
            retrieved_chunks = self.doc_retriever_tool(query).raw_output
        ctx.data["stored_chunks"].extend(retrieved_chunks)

        # synthesize an answer given the query to return to the LLM.
        response = self.summarizer.synthesize(query, nodes=retrieved_chunks)
        self.memory.put(
            ChatMessage(
                role="tool",
                content=str(response),
                additional_kwargs={
                    "tool_call_id": ev.tool_call.tool_id,
                    "name": ev.tool_call.tool_name,
                },
            )
        )

        # send input event back with updated chat history
        return InputEvent(input=self.memory.get())

    @step(pass_context=True)
    async def generate_report(
        self, ctx: Context, ev: ReportGenerationEvent
    ) -> StopEvent:
        """Generate report."""
        # given all the context, generate query
        response = self.report_gen_summarizer.synthesize(
            ctx.data["query"], nodes=ctx.data["stored_chunks"]
        )

        return StopEvent(result={"response": response})

In [ ]:
agent = ReportGenerationAgent(
    chunk_retriever_tool,
    doc_retriever_tool,
    llm=llm,
    report_gen_sllm=report_gen_sllm,
    verbose=True,
    timeout=120.0,
)

In [ ]:
ret = await agent.run(
    input="Tell me about the top-level assets and liabilities for Tesla in 2021, and compare it against those of Apple in 2021. Which company is doing better?"
)

Running step prepare_chat_history
Step prepare_chat_history produced event InputEvent
Running step handle_llm_input
Tool call: tool_id='call_p2VqKBvTaQ84lSQ8oU59cbPo' tool_name='doc_retriever_fn' tool_kwargs={'query': 'Tesla 2021 financial statements assets liabilities'}
Step handle_llm_input produced event DocRetrievalEvent
Running step handle_retrieval
Step handle_retrieval produced event InputEvent
Running step handle_llm_input
Tool call: tool_id='call_hY4phZeNHTLkkDWqjHCk9lA1' tool_name='doc_retriever_fn' tool_kwargs={'query': 'Apple 2021 financial statements assets liabilities'}
Step handle_llm_input produced event DocRetrievalEvent
Running step handle_retrieval
Step handle_retrieval produced event InputEvent
Running step handle_llm_input
Step handle_llm_input produced event ReportGenerationEvent
Running step generate_report
Step generate_report produced event StopEvent


In [ ]:
ret["response"].response.render()

In 2021, Tesla and Apple reported their top-level assets and liabilities in their respective annual reports. Here is a detailed comparison of their financial positions for the year 2021.

,Category,Tesla (in millions),Apple (in millions)
0,Total Assets,"$62,131","$351,002"
1,Total Liabilities,"$30,548","$287,912"


Tesla's total assets in 2021 amounted to $62.131 billion, while its total liabilities were $30.548 billion. In comparison, Apple's total assets were significantly higher at $351.002 billion, with total liabilities of $287.912 billion.

,Company,Total Assets (in billions),Total Liabilities (in billions)
0,Tesla,$62.131,$30.548
1,Apple,$351.002,$287.912


When comparing the financial positions of Tesla and Apple in 2021, it is evident that Apple had a much stronger financial position with significantly higher total assets and liabilities. Apple's total assets were approximately 5.65 times greater than Tesla's, and its total liabilities were about 9.42 times higher than Tesla's. This indicates that Apple had more resources at its disposal and a larger scale of operations compared to Tesla. Therefore, in terms of financial metrics, Apple was doing better than Tesla in 2021.

In [ ]:
ret = await agent.run(
    input="Tell me about the gross margin breakdown of Apple 2020-2023."
)

Running step prepare_chat_history
Step prepare_chat_history produced event InputEvent
Running step handle_llm_input
Tool call: tool_id='call_Xqx4IUccHisGohNp4wQRWYyE' tool_name='chunk_retriever_fn' tool_kwargs={'query': 'Apple gross margin breakdown 2020'}
Step handle_llm_input produced event ChunkRetrievalEvent
Running step handle_retrieval
Step handle_retrieval produced event InputEvent
Running step handle_llm_input
Tool call: tool_id='call_wTlQF8mnKlSpOsGzVAQm8scx' tool_name='chunk_retriever_fn' tool_kwargs={'query': 'Apple gross margin breakdown 2021'}
Step handle_llm_input produced event ChunkRetrievalEvent
Running step handle_retrieval
Step handle_retrieval produced event InputEvent
Running step handle_llm_input
Tool call: tool_id='call_Xh1liurupCNVIXavyS4v03gQ' tool_name='chunk_retriever_fn' tool_kwargs={'query': 'Apple gross margin breakdown 2022'}
Step handle_llm_input produced event ChunkRetrievalEvent
Running step handle_retrieval
Step handle_retrieval produced event InputEv

In [ ]:
ret["response"].response.render()

The gross margin breakdown of Apple Inc. from 2020 to 2023 provides insights into the company's profitability across its product and service lines. The gross margin is a critical financial metric that indicates the difference between sales and the cost of goods sold, reflecting the efficiency of production and pricing strategies.

,,2023,2022,2021,2020
0,Products,"$108,803","$114,728","$105,126","$69,461"
1,Services,"$60,345","$56,054","$47,710","$35,495"
2,Total gross margin,"$169,148","$170,782","$152,836","$104,956"


The gross margin for products and services has shown a consistent increase over the years, with a notable rise in services gross margin. This indicates a growing contribution of services to the overall profitability of the company.

,,2023,2022,2021,2020
0,Products,36.5%,36.3%,35.3%,31.5%
1,Services,70.8%,71.7%,69.7%,66.0%
2,Total gross margin percentage,44.1%,43.3%,41.8%,38.2%


The gross margin percentage for both products and services has also increased, reflecting improved efficiency and cost management. The services segment, in particular, has a significantly higher gross margin percentage compared to products, highlighting its higher profitability.

In 2023, the gross margin for products decreased compared to 2022 due to the weakness in foreign currencies relative to the U.S. dollar and lower product volumes. However, the gross margin percentage increased due to cost savings and a different product mix. For services, the gross margin increased due to higher net sales, although the gross margin percentage slightly decreased due to higher service costs and the impact of foreign currency fluctuations.

Overall, the data indicates that while the products segment faces challenges related to currency fluctuations and volume changes, the services segment continues to grow and contribute significantly to Apple's profitability.

In [ ]:
ret = await agent.run(
    input="Give me a condensed summary of Tesla in 2023"
)

Running step prepare_chat_history
Step prepare_chat_history produced event InputEvent
Running step handle_llm_input
Tool call: tool_id='call_ZupQtlUffBFo2oTF72ngp8aH' tool_name='doc_retriever_fn' tool_kwargs={'query': 'Tesla summary 2023'}
Step handle_llm_input produced event DocRetrievalEvent
Running step handle_retrieval
Step handle_retrieval produced event InputEvent
Running step handle_llm_input
Step handle_llm_input produced event ReportGenerationEvent
Running step generate_report
Step generate_report produced event StopEvent


In [ ]:
ret["response"].response.render()

Tesla, Inc. is a Delaware corporation headquartered in Austin, Texas, that designs, develops, manufactures, sells, and leases high-performance fully electric vehicles and energy generation and storage systems. The company operates through two main segments: automotive and energy generation and storage.

In 2023, Tesla produced 1,845,985 consumer vehicles and delivered 1,808,581 consumer vehicles. The company manufactures five different consumer vehicles: Model 3, Model Y, Model S, Model X, and Cybertruck. Additionally, Tesla began early production and deliveries of the Tesla Semi, a commercial electric vehicle.

,Metric,Value
0,Vehicles Produced,"1,845,985"
1,Vehicles Delivered,"1,808,581"


Tesla's energy generation and storage segment includes products like Powerwall and Megapack, which are lithium-ion battery energy storage products. In 2023, Tesla deployed 14.72 GWh of energy storage products and 223 megawatts of solar energy systems.

,Metric,Value
0,Energy Storage Products Deployed,14.72 GWh
1,Solar Energy Systems Deployed,223 MW


...

,Metric,Value
0,Total Revenues,$96.77 billion
1,Net Income,$15.00 billion


Tesla's cash and cash equivalents and investments totaled $29.09 billion at the end of 2023, representing an increase of $6.91 billion from the end of 2022. The company continues to invest in expanding its manufacturing capacity, developing new products, and enhancing its service and charging infrastructure.

Tesla operates several manufacturing facilities globally, including Gigafactory Texas, Fremont Factory, Gigafactory Nevada, Gigafactory Berlin-Brandenburg, Gigafactory Shanghai, and Gigafactory New York. The company is also constructing a new Gigafactory in Monterrey, Mexico.

,Facility,Location,Status
0,Gigafactory Texas,"Austin, Texas",Owned
1,Fremont Factory,"Fremont, California",Owned
2,Gigafactory Nevada,"Sparks, Nevada",Owned
3,Gigafactory Berlin-Brandenburg,"Grunheide, Germany",Owned
4,Gigafactory Shanghai,"Shanghai, China",Owned
5,Gigafactory New York,"Buffalo, New York",Leased
6,Megafactory,"Lathrop, California",Leased


Tesla's mission is to accelerate the world's transition to sustainable energy. The company emphasizes performance, attractive styling, and safety in its products while striving to lower the cost of ownership for its customers. Tesla continues to develop full self-driving technology and aims to establish an autonomous Tesla ride-hailing network in the future.